# P4

## Récupération des données

In [1]:
import pandas as pd
import numpy as np
import time
import os

# Option pour ne pas tout charger
ALLFILES = False
CT_DIR_DATA = 'C:\Work\OpenClassrooms\data\P4/'

filename = '2016_'
# Récupération du 1er fichier
dataraw = pd.read_csv(CT_DIR_DATA + filename + '01_ready.csv', sep="\t", encoding='utf-8')
print(dataraw.shape)
print(dataraw.columns)

if ALLFILES:
    # Concaténation de toutes les données
    for i in range(2, 13):
        n = filename + '%0*d' % (2, i) + '_ready.csv'
        #print('reading', n, '...')
        f = pd.read_csv(n, sep="\t", encoding='utf-8')
        print(n, f.shape)
        print(f.columns)
        dataraw = dataraw.append(f)
        print(dataraw.shape)
else:
    dataraw = dataraw.sample(1000)

(433298, 9)
Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'UNIQUE_CARRIER',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME', 'ARR_DELAY', 'HDAYS',
       'DEP_HOUR'],
      dtype='object')


In [2]:
# Méthodes d'import export
import pickle
from sklearn.externals import joblib
CT_DIR = '../flightdelay/save/'

def save_obj(obj, name):
    fn = CT_DIR + name + '.pkl'
    try:
        os.remove(fn)
    except OSError:
        pass
    with open(fn, 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    print(fn, 'saved')

def load_obj(name):
    with open(CT_DIR + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
def save_sklearn_obj(obj, name):
    fn = CT_DIR + name + '.pkl'
    try:
        os.remove(fn)
    except OSError:
        pass
    joblib.dump(obj, fn)
    print(fn, 'saved')

def load_sklearn_obj(name):
    return joblib.load(CT_DIR + name + '.pkl')

## Préparation au calcul

In [3]:
# Conversion pour les noms des colonnes
dataraw[['ORIGIN_AIRPORT_ID']] = dataraw[['ORIGIN_AIRPORT_ID']].astype(np.int64)

In [4]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def TestModel(p_df, c):

    # On fait un choix de colonnes restreint en pensant à ce que l'utilisateur pourra renseigner dans la page web
    # Les numériques
    scalingDF = p_df[['HDAYS']].astype('float')
    # Les catégories
    categDF = p_df[['MONTH', 'ORIGIN_AIRPORT_ID', 'DEP_HOUR', 'DAY_OF_WEEK']]
    
    y_final = p_df['ARR_DELAY'].values
    y_final.mean()        
    
    # Binarisation en dummies pour garder la maitrise des noms des colonnes
    categDF_encoded = pd.get_dummies(categDF.astype(np.str))

    # Concaténation
    x_final = pd.concat([scalingDF, categDF_encoded], axis=1)

    # Sauvegarde de la liste des colonnes pour le web
    x_save = x_final[x_final.HDAYS==-1]
    save_obj(x_save, 'model_columns_'+c)

    print('Done!')

## GO

In [5]:
lst_carriers = list(sorted(set(dataraw['UNIQUE_CARRIER'])))
for c in lst_carriers:
    print(c)
    df = dataraw[dataraw.UNIQUE_CARRIER == c]
    print (df.shape)
    #df.to_csv('C:\Work\OpenClassrooms\data\P4/2016_' + c + '.csv', sep="\t", encoding='utf-8', index=False)
    
    TestModel(df, c)
    break
    

AA
(172, 9)
../flightdelay/save/test_AA.pkl saved
Done!
